In [1]:
import json, glob, pathlib
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings   # o HuggingFace
from langchain.vectorstores import FAISS            # o Chroma, Milvus…
from sentence_transformers import SentenceTransformer


/Users/maxi/miniforge3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
JSONS_FOLDER = "datasets/fallos_json"
OUTPUT_FOLDER = "datasets/fallos_vectorstore"

EMBEDDINGS_MODEL = "mixedbread-ai/mxbai-embed-large-v1"
CHUNK_SIZE = 1024
CHUNK_OVERLAP = 128

In [3]:

def load_cases(folder=JSONS_FOLDER):
    """Devuelve una lista de dicts {'case_id', 'section', 'text'}."""
    rows = []
    for path in pathlib.Path(folder).glob("*.json"):
        data = json.loads(path.read_text())[0]       # un fallo por archivo
        case = path.stem                             # ej. 8104
        for section, lines in data["CONTENIDO"].items():
            # Unimos las líneas originales
            section_txt = "\n".join(lines).strip()
            rows.append({"case_id": case,
                         "section": section,
                         "text": section_txt})
    return rows

In [4]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,             
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", " ", ""]
)

docs = []
for row in load_cases():
    for chunk_id, chunk in enumerate(splitter.split_text(row["text"])):
        docs.append(
            Document(
                page_content=chunk,
                metadata={
                    "case_id": row["case_id"],
                    "section": row["section"],
                    "chunk_id": chunk_id
                }
            )
        )

print(f"[INFO] Total chunks: {len(docs)}")

[INFO] Total chunks: 253


In [5]:
# tutorial
# embeddings_wrapper.py
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.docstore.document import Document

class LangchainSentenceTransformer:
    """
    Wrapper simple para usar un modelo open-source de Sentence-Transformers
    dentro de LangChain (sólo lo que FAISS necesita).
    """

    def __init__(self, model_name="mixedbread-ai/mxbai-embed-large-v1"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model  = SentenceTransformer(model_name).to(self.device)

        # prefijo opcional sugerido por el paper "bge" / MixedBread
        self.query_prefix = "Represent this sentence for searching relevant passages: "

        # almacena docs limpios para referencia
        self._cleaned_documents: list[Document] = []

    # --- Métodos requeridos por LangChain -----------------
    def embed_documents(self, documents):
        """
        Recibe una lista de langchain.Document y devuelve (texts, np.ndarray[n_docs, dim])
        """
        # 1) limpieza rápida opcional  ──> adaptala si lo necesitás
        bad = ["Atomic Structure Representation", "Logical path\nD\nF\nC\nA\nE"]
        filtered = [
            Document(page_content=doc.page_content.strip())
            for doc in documents
            if doc.page_content.strip() and not any(b in doc.page_content for b in bad)
        ]
        self._cleaned_documents = filtered

        # 2) embed
        texts = [d.page_content for d in filtered]
        vectors = self.model.encode(
            texts,
            show_progress_bar=True,
            convert_to_numpy=True,
            device=self.device,
        )
        return texts, vectors

    def embed_query(self, query: str):
        """
        Embedding de una sola consulta (devuelve np.ndarray[dim])
        """
        vec = self.model.encode(
            [self.query_prefix + str(query)],
            convert_to_numpy=True,
            device=self.device,
        )[0]
        return vec

    # hace al wrapper "callable" dentro de LangChain si se lo pasa como función
    def __call__(self, text: str):
        return self.embed_query(text)


In [ ]:
embedder           = LangchainSentenceTransformer(
                        model_name="mixedbread-ai/mxbai-embed-large-v1"  
                    )
texts, vectors     = embedder.embed_documents(docs)   # devuelve np.ndarray
pairs              = list(zip(texts, vectors))

In [ ]:
# Ejemplo de búsqueda
retriever = vectordb.as_retriever(search_k=5)
query = "constitución en mora artículo 1849"
for doc in retriever.get_relevant_documents(query):
    print(doc.metadata)   # -> {'case_id': '8344', 'section': 'GISELA N. SCHUMACHER DIJO', ...}
